# Semi-supervised Community Detection using Graph Neural Networks

Almost every computer 101 class starts with a "Hello World" example. Like MNIST for deep learning, in graph domain we have the Zachary's Karate Club problem. The karate club is a social network that includes 34 members and documents pairwise links between members who interact outside the club. The club later divides into two communities led by the instructor (node 0) and the club president (node 33). The network is visualized as follows with the color indicating the community.

<img src='../asset/karat_club.png' align='center' width="400px" height="300px" />

In this tutorial, you will learn:

* Formulate the community detection problem as a semi-supervised node classification task.
* Build a GraphSAGE model, a popular Graph Neural Network architecture proposed by [Hamilton et al.](https://arxiv.org/abs/1706.02216)
* Train the model and understand the result.

In [12]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools

## Community detection as node classification

The study of community structure in graphs has a long history. Many proposed methods are *unsupervised* (or *self-supervised* by recent definition), where the model predicts the community labels only by connectivity. Recently, [Kipf et al.,](https://arxiv.org/abs/1609.02907) proposed to formulate the community detection problem as a semi-supervised node classification task. With the help of only a small portion of labeled nodes, a GNN can accurately predict the community labels of the others.

In this tutorial, we apply Kipf's setting to the Zachery's Karate Club network to predict the community membership, where only the labels of a few nodes are used.

We first load the graph and node labels as is covered in the [last session](./1_load_data.ipynb). Here, we have provided you a function for loading the data.

In [13]:
from tutorial_utils import load_zachery

# ----------- 0. load graph -------------- #
g = load_zachery()
print(g)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})


In the original Zachery's Karate Club graph, nodes are feature-less. (The `'Age'` attribute is an artificial one mainly for tutorial purposes). For feature-less graph, a common practice is to use an embedding weight that is updated during training for every node.

We can use PyTorch's `Embedding` module to achieve this.

In [14]:
# ----------- 1. node features -------------- #
node_embed = nn.Embedding(g.number_of_nodes(), 5)  # Every node has an embedding of size 5.
inputs = node_embed.weight                         # Use the embedding weight as the node features.
nn.init.xavier_uniform_(inputs)
print(inputs)

Parameter containing:
tensor([[ 0.0198, -0.2025, -0.2163,  0.3326, -0.0799],
        [ 0.2821,  0.2558,  0.1610, -0.3752,  0.3142],
        [ 0.1328, -0.0017, -0.1863, -0.2835,  0.1020],
        [ 0.1286,  0.1920, -0.0098, -0.0716,  0.1045],
        [-0.1406, -0.2568,  0.3155, -0.2180,  0.0859],
        [-0.2981,  0.1447,  0.0224,  0.0149, -0.1057],
        [ 0.2976,  0.2175,  0.3228,  0.3839, -0.2936],
        [ 0.2523,  0.1296, -0.0392,  0.1322,  0.1846],
        [-0.3500, -0.3145, -0.3418, -0.1453, -0.2096],
        [ 0.0257,  0.2476, -0.0852,  0.3569,  0.2647],
        [ 0.1103, -0.3191,  0.1200,  0.3574,  0.2689],
        [ 0.1190, -0.2395,  0.2617,  0.3697, -0.1608],
        [ 0.0281,  0.1334,  0.0420, -0.2567, -0.1763],
        [ 0.0019,  0.0061,  0.3576,  0.1254, -0.1403],
        [-0.2442, -0.2989, -0.0877,  0.2636, -0.0194],
        [-0.1751,  0.0462,  0.0049,  0.1428, -0.1204],
        [ 0.1387,  0.3585,  0.3322,  0.3719, -0.0400],
        [ 0.1394, -0.1515, -0.0665,  0.0098

The community label is stored in the `'club'` node feature (0 for instructor, 1 for club president). Only nodes 0 and 33 are labeled.

In [24]:
import random
import numpy as np
random.seed(0)
labels = g.ndata['club']
print('#nodes:', len(labels))
train_nodes = np.unique([0, 33] + random.sample(range(len(labels)), 3))
test_nodes = np.delete(np.arange(len(labels)), train_nodes)
print('#labeled nodes:', len(train_nodes))
print('Labels', labels[train_nodes])

#nodes: 34
#labeled nodes: 5
Labels tensor([0, 0, 1, 1, 1])


## Message passing and GNNs

DGL follows the *message passing paradigm* inspired by the Message Passing Neural Network proposed by [Gilmer et al.](https://arxiv.org/abs/1704.01212) Essentially, they found many GNN models can fit into the following framework:

$$
m_{u\sim v}^{(l)} = M^{(l)}\left(h_v^{(l-1)}, h_u^{(l-1)}, e_{u\sim v}^{(l-1)}\right)
$$

$$
m_{v}^{(l)} = \sum_{u\in\mathcal{N}(v)}m_{u\sim v}^{(l)}
$$

$$
h_v^{(l)} = U^{(l)}\left(h_v^{(l-1)}, m_v^{(l)}\right)
$$

where DGL calls $M^{(l)}$ the *message function* and $\sum$ the *reduce function*.  Note that $\sum$ here can represent any function and is not necessarily a summation.

## Define a GraphSAGE model

Our model consists of two layers, each computes new node representations by aggregating neighbor information. The equations are:

$$
h_{\mathcal{N}(v)}^k\leftarrow \text{AGGREGATE}_k\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \sigma\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$


You can see that message passing is directional: the message sent from one node $u$ to other node $v$ is not necessarily the same as the other message sent from node $v$ to node $u$ in the opposite direction.

DGL graphs provide two members `srcdata` and `dstdata` for the purpose of message passing.  You first put the input node features in `srcdata`.  After you perform message passing, you can retrieve the result of message passing from `dstdata`.

<div class="alert alert-info">
    <b>Note: </b>In full graph message passing, both the input nodes and the output nodes are the full node set.  Therefore, <code>srcdata</code> and <code>dstdata</code> in homogeneous graph (i.e. with only one node type and one edge type) are identical to <code>ndata</code>.  All graphs in this tutorial section are homogeneous.
</div>

In [25]:
import dgl.function as fn

class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.
    
    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h):
        """Forward computation
        
        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.srcdata['h'] = h
            # update_all is a message passing API.
            # equation 1 and 2
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_neigh'))
            # equation 3
            h_neigh = g.dstdata['h_neigh']
            h_total = torch.cat([h, h_neigh], dim=1)
            return F.relu(self.linear(h_total))

Implement a multi-layer GraphSage model.

<img src='../asset/multi_layer.png' align='center' width="600px" height="450px" />


In [26]:
# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# Create the model with given dimensions 
# input layer dimension: 5, node embeddings
# hidden layer dimension: 16
# output layer dimension: 2, the two classes, 0 and 1
net = GraphSAGE(5, 16, 2)

In [27]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(net.parameters(), node_embed.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    logits = net(g, inputs)
    
    # compute loss
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_nodes], labels[train_nodes])
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    all_logits.append(logits.detach())
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.7718736529350281
In epoch 5, loss: 0.5637348890304565
In epoch 10, loss: 0.40559178590774536
In epoch 15, loss: 0.2468864917755127
In epoch 20, loss: 0.11854676902294159
In epoch 25, loss: 0.04251787066459656
In epoch 30, loss: 0.012699922546744347
In epoch 35, loss: 0.004077625460922718
In epoch 40, loss: 0.0016410115640610456
In epoch 45, loss: 0.0008527038735337555
In epoch 50, loss: 0.0005473890341818333
In epoch 55, loss: 0.0004072349111083895
In epoch 60, loss: 0.0003339156974107027
In epoch 65, loss: 0.0002915450604632497
In epoch 70, loss: 0.0002648533845786005
In epoch 75, loss: 0.00024674052838236094
In epoch 80, loss: 0.00023348924878519028
In epoch 85, loss: 0.00022319315758068115
In epoch 90, loss: 0.0002145891048712656
In epoch 95, loss: 0.00020715282880701125


In [28]:
# ----------- 5. check results ------------------------ #
pred = torch.argmax(logits, axis=1)
print('Accuracy', (pred == labels)[test_nodes].sum().item() / len(pred))

Accuracy 0.7941176470588235


## Implement GraphSage with nn modules

DGL provides implementation of many popular neighbor aggregation modules. . They all can be invoked easily with one line of code. See the full list of supported [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv).

In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, num_classes, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# Create the model with given dimensions 
# input layer dimension: 5, node embeddings
# hidden layer dimension: 16
# output layer dimension: 2, the two classes, 0 and 1
net = GraphSAGE(5, 16, 2)

## Exercise

Play with the GNN models by using other [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv). For example, how about graph attention networks?